In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# 中選會資料爬取

In [ ]:
interest = ['01','02','03','04','05','06','07','08','09','10','11','12']
#爬取各行政區市長選舉的資料
def getref(url):
    res = requests.get(url)       
    soup=BeautifulSoup(res.text,'lxml')
    dfs = pd.read_html(res.text) 
    votes=dfs[0]
    votes.drop([0,1,3,6],axis=1,inplace=True)
    votes.drop([0,2,3,4,6,7],axis=0,inplace=True)
    votes.columns=votes.loc[1]
    votes.drop([1],axis=0,inplace=True)
    area=soup.select_one('b').text
    votes['投票地區']=area
    return votes
#爬取各行政區公投第10案的資料
def getref2(url):
    res = requests.get(url)       
    soup=BeautifulSoup(res.text,'lxml')
    dfs = pd.read_html(res.text) 
    votes=dfs[2]
    votes.columns=votes.loc[1]
    votes.drop([0,1,3,4],axis=0,inplace=True)
    votes.reset_index(drop=True,inplace=True)
    totalvotes = dfs[3]
    totalvotes.columns=totalvotes.loc[0]
    totalvotes.drop([0,2,3],axis=0,inplace=True)
    totalvotes.reset_index(drop=True,inplace=True)
    m = pd.concat([votes,totalvotes],axis=1)
    area=soup.select_one('b').text
    m['投票地區']=area
    return m
#爬取各投票所的詳細資料(台北市共約1500個投票所)
def detail(domain,tail,newdomain,getref):
    results2 = []
    k = 0
    for ele in interest:
        results = []
        driver = webdriver.Chrome('D:\PythonPractice\chromedriver')
        driver.get(domain + ele + tail)
        soup = BeautifulSoup(driver.page_source,'lxml')
        links = soup.select('option')
        for i in links:
            try:
                results.append(getref(newdomain + i.get('value')))
            except:
                print(newdomain + i.get('value'))
        results = pd.concat(results)
        results2.append(results)
    return results2
#先做關於市長選舉
domain = 'http://vote.2018.nat.gov.tw/pc/zh_TW/TC/n6300000'
tail = '00000000.html'
domain2 = 'http://vote.2018.nat.gov.tw/pc/zh_TW/TC/'
#將各行政區市長選舉結果總計及各投票所資料抓取並存入vote.xlsx
results = detail(domain,tail,domain2,getref)
k = 1
writer = pd.ExcelWriter('D:\PythonPractice/vote.xlsx')
#按行政區分別存為12個不同的sheet
for ele in results:
    ele.to_excel(writer,sheet_name='vote{}'.format(k))
    k = k+1
writer.save()
#再做公投第10案
domain3 = 'http://referendum.2018.nat.gov.tw/pc/zh_TW/04/6300000'
tail2 = '00000000.html'
domain4 = 'http://referendum.2018.nat.gov.tw/pc/zh_TW/04/'
#將各行政區公投第10案結果總計及各投票所資料抓取並存入policy.xlsx
results = detail(domain3,tail2,domain4,getref2)
k = 1
writer = pd.ExcelWriter('D:\PythonPractice/policy.xlsx')
#按行政區分別存為12個不同的sheet
for ele in results:
    ele.to_excel(writer,sheet_name='{}'.format(k))
    k = k+1
writer.save()

# 模型計算&檢定

In [ ]:
df = pd.read_excel('D:\PythonPractice\整理後資料.xlsx')
x1 = np.transpose(np.array([np.ones(len(var1)),df['65歲以上比例'],df['大學以上比例']]))
x2 = np.transpose(np.array([np.ones(len(var2)),df['50歲以上比例'],df['大學以上比例']]))
y1 = np.array(df['柯得票率']).reshape(12,1)
y2 = np.array(df['10同意率']) .reshape(12,1)
#定義函數white,丟入x,y得到White檢定統計量
def white(x,y):
    def beta(x,y):
        a = np.linalg.inv(np.dot(np.transpose(x),x))
        b = np.dot(np.transpose(x),y)
        c = np.dot(a,b) .reshape(len(np.transpose(x)),1)
        return c
    betaOLS = beta(x,y)
    ehat = y - np.dot(x,betaOLS)
    ehatsq = np.square(ehat)
#建置輔助迴歸變數矩陣
    aux = []
    for i in range(1,len(np.transpose(x))):
        newx = np.multiply(np.transpose(x)[i],np.transpose(x)[i])
        aux.append(newx)
    for i in range(1,len(np.transpose(x))):
        for j in range(i+1,len(np.transpose(x))):
            try:
                newx = np.multiply(np.transpose(x)[i],np.transpose(x)[j])
                aux.append(newx)
            except:
                break
    aux = np.transpose(np.array(aux))
    newaux = np.hstack((x,aux))
    auxbeta = beta(newaux,ehatsq)
#計算輔助迴歸式的TSS及RSS
    TSS = 0
    for i in ehatsq:
        TSS = TSS + np.square(i - np.mean(ehatsq))
    RSS = 0
    for i in np.dot(newaux,auxbeta):
        RSS = RSS + np.square(i - np.mean(np.dot(newaux,auxbeta)))
    return print('White檢定統計量為{},且chi-squared自由度為{}'.format(len(y)*RSS/TSS,len(np.transpose(newaux))-1))

In [ ]:
#得到model1及model2的檢定統計量
print(white(x1,y1))
print(white(x2,y2))

In [ ]:
#計算OLS下的beta
def beta(x,y):
    a = np.linalg.inv(np.dot(np.transpose(x),x))
    b = np.dot(np.transpose(x),y)
    c = np.dot(a,b) .reshape(len(np.transpose(x)),1)
    return c
#得到兩模型的beta矩陣
beta1 = beta(x1,y1)
beta2 = beta(x2,y2)
# beta顯著性t檢定
def betatest(R,r,x,y):
    def beta(x,y):
        a = np.linalg.inv(np.dot(np.transpose(x),x))
        b = np.dot(np.transpose(x),y)
        c = np.dot(a,b) .reshape(len(np.transpose(x)),1)
        return c
    betaOLS = beta(x,y)
    a = np.dot(R,betaOLS) - r
    b = (np.dot(np.transpose(y - np.dot(x,betaOLS)),y - np.dot(x,betaOLS))/(len(y)- len(np.transpose(x))))**0.5
    c = np.dot(np.dot(R,np.linalg.inv(np.dot(np.transpose(x),x))),np.transpose(R))**0.5
    return print('t檢定統計量為{}'.format(a/(b*c)))
#檢測beta顯著性
R1 = np.array([0,1,0]).reshape(1,3)
R2 = np.array([0,0,1]).reshape(1,3)
r = np.zeros(shape=(1,1))
betatest(R1,r,x1,y1)
t檢定統計量為[[-1.39233134]]
betatest(R2,r,x1,y1)
t檢定統計量為[[-1.78211522]]
betatest(R1,r,x2,y2)
t檢定統計量為[[-1.19168418]]
betatest(R2,r,x2,y2)
t檢定統計量為[[0.71632948]]
#整體F檢定
def Ftest(R,r,x,y):
    def beta(x,y):
        a = np.linalg.inv(np.dot(np.transpose(x),x))
        b = np.dot(np.transpose(x),y)
        c = np.dot(a,b) .reshape(len(np.transpose(x)),1)
        return c
    betaOLS = beta(x,y)
    a = np.dot(R,betaOLS) - r
    b = np.dot(np.transpose(y - np.dot(x,betaOLS)),y - np.dot(x,betaOLS))/(len(y)- len(np.transpose(x)))
    c = np.linalg.inv(np.dot(np.dot(R,np.linalg.inv(np.dot(np.transpose(x),x))),np.transpose(R)))
    return print('F檢定統計量為{}'.format(np.dot(np.dot(np.transpose(a),c),a)/(len(R)*b)))

In [ ]:
#檢定整體顯著性
R3 = np.array([[0,1,0],[0,0,1]])
r2 = np.zeros(shape=(2,1))
print(Ftest(R3,r2,x1,y1))
print(Ftest(R3,r2,x2,y2))